<a href="https://colab.research.google.com/github/dustinmaurer/datascience_tutorials/blob/main/20250217_recommendations_associationRules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('Uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ml-latest-small.zip to ml-latest-small (1).zip
Uploaded file "ml-latest-small (1).zip" with length 978202 bytes


In [7]:
fn = "ml-latest-small.zip"
import zipfile
with zipfile.ZipFile(fn, 'r') as zip_ref:
    zip_ref.extractall()

In [10]:
import pandas as pd

df = pd.read_csv('/content/ml-latest-small/ratings.csv')
print(df.head())


   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [13]:
# prompt: show the head of the movies.csv df
movies_df = pd.read_csv('/content/ml-latest-small/movies.csv')
print(movies_df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [12]:
# prompt: limit the df to userId <= 10. give the total counts for each user for each rating
limited_df = df[df['userId'] <= 10]
rating_counts = limited_df.groupby(['userId', 'rating']).size().unstack(fill_value=0)
rating_counts

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,1,0,5,0,26,0,76,0,124
2,0,0,0,1,1,4,4,9,4,6
3,20,0,0,1,0,1,1,1,5,10
4,0,23,0,26,0,39,0,64,0,64
5,0,1,0,3,0,17,0,13,0,10
6,0,7,0,13,0,152,0,102,0,40
7,6,12,12,11,7,16,18,28,29,13
8,0,1,0,3,0,21,0,12,0,10
9,0,6,0,6,0,12,0,14,0,8


In [23]:
# for every user, limit to the most recent 10 reviews. of those ratings, keep only the top half of those.

# Group the DataFrame by userId and sort by timestamp in descending order
grouped_df = (
    df.sort_values(['userId', 'timestamp'], ascending=[True, False])
      .groupby('userId', group_keys=False)
      .head(10)
)

# Reset the index
grouped_df = grouped_df.reset_index(drop=True)

# Group by userId again and find the median rating
median_ratings = grouped_df.groupby('userId')['rating'].median()

# Merge the median ratings back into the DataFrame
grouped_df = grouped_df.merge(median_ratings, on='userId', suffixes=('', '_median'))

# Filter out ratings below the median
filtered_df = grouped_df[grouped_df['rating'] >= grouped_df['rating_median']]

# Display the result
filtered_df


,userId,movieId,rating,timestamp,rating_median
3,1,553,5.0,964984153,4.5
5,1,157,5.0,964984100,4.5
7,1,1298,5.0,964984086,4.5
8,1,3053,5.0,964984086,4.5
9,1,3448,5.0,964984054,4.5
...,...,...,...,...,...
6093,610,70,4.0,1495959282,4.0
6095,610,3727,4.0,1495959265,4.0
6096,610,2901,4.0,1495959241,4.0
6098,610,63826,4.0,1495959194,4.0


In [25]:
# prompt: for every userId, find every pair of movieId. the result should have 3 columns, userId, movie1, and movie2

from itertools import combinations

def find_movie_pairs(df):
    movie_pairs = []
    for user_id, group in df.groupby('userId'):
        movie_ids = group['movieId'].tolist()
        for pair in combinations(movie_ids, 2):
            movie_pairs.append([user_id, pair[0], pair[1]])
    return pd.DataFrame(movie_pairs, columns=['userId', 'movie1', 'movie2'])

movie_combinations = find_movie_pairs(filtered_df)
movie_combinations


,userId,movie1,movie2
0,1,553,157
1,1,553,1298
2,1,553,3053
3,1,553,3448
4,1,157,1298
...,...,...,...
12683,610,3727,63826
12684,610,3727,3016
12685,610,2901,63826
12686,610,2901,3016
